---
title: "08-卡方检验与Fisher精确检验"
author: "Simon Zhou"
date: "2025-05-06"
format: 
    html:
        code-fold: false
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 2
---

In [2]:
import stata_setup
stata_setup.config('C:/Program Files/Stata18', 'mp', splash=False)

## 数据导入

此章节使用 Stata 自带的 1988 年 美国的 National Longitudinal Study of Young Women Data 进行分析和示例。

In [3]:
%%stata
sysuse nlsw88.dta, clear

(NLSW, 1988 extract)


## $\chi^2$ test

卡方检验($\chi^2$)，主要用于检验两个或多个分类变量之间的关联性。它通过比较实际观察频数与期望频数之间的差异来判断变量是否独立，从而揭示不同类别之间是否存在统计学上的显著关系。

基本语法：

```stata
tabulate var1 var2,chi2
```

`tabulate` 可以简写为 `tab`，后续都用 `tab` 替代 `tabulate` 

In [ ]:
%%stata
tab race married, chi2


           |        Married
      Race |    Single    Married |     Total
-----------+----------------------+----------
     White |       487      1,150 |     1,637 
     Black |       309        274 |       583 
     Other |         8         18 |        26 
-----------+----------------------+----------
     Total |       804      1,442 |     2,246 

          Pearson chi2(2) = 101.4215   Pr = 0.000


### 查看每一个单元格对于 $\chi^2$ 检验的贡献:

语法：

```stata
tab var1 var2, cchi2 chi2
```

In [ ]:
%%stata
tab race married, cchi2 chi2


+-------------------+
| Key               |
|-------------------|
|     frequency     |
| chi2 contribution |
+-------------------+

           |        Married
      Race |    Single    Married |     Total
-----------+----------------------+----------
     White |       487      1,150 |     1,637 
           |      16.7        9.3 |      26.0 
-----------+----------------------+----------
     Black |       309        274 |       583 
           |      48.2       26.9 |      75.1 
-----------+----------------------+----------
     Other |         8         18 |        26 
           |       0.2        0.1 |       0.3 
-----------+----------------------+----------
     Total |       804      1,442 |     2,246 
           |      65.1       36.3 |     101.4 

          Pearson chi2(2) = 101.4215   Pr = 0.000


### 查看期望频数（理论频数）

语法：

```stata
tab var1 var2, chi2 expected
```

In [ ]:
%%stata
tab race married, chi2 expected


+--------------------+
| Key                |
|--------------------|
|     frequency      |
| expected frequency |
+--------------------+

           |        Married
      Race |    Single    Married |     Total
-----------+----------------------+----------
     White |       487      1,150 |     1,637 
           |     586.0    1,051.0 |   1,637.0 
-----------+----------------------+----------
     Black |       309        274 |       583 
           |     208.7      374.3 |     583.0 
-----------+----------------------+----------
     Other |         8         18 |        26 
           |       9.3       16.7 |      26.0 
-----------+----------------------+----------
     Total |       804      1,442 |     2,246 
           |     804.0    1,442.0 |   2,246.0 

          Pearson chi2(2) = 101.4215   Pr = 0.000


## Fisher's exact test

当列联表中理论频数(期望频数)<5时，我们可以增加样本量、删去理论频数太少的行或列、或者合并某些行或列。

当然也可以使用Fisher精确检验来检验，并且任何样本量都可以使用Fisher精确检验。

语法：

```stata
tab var1 var2, exact
```

In [ ]:
%%stata
tab race married, exact


Enumerating sample-space combinations:
stage 3:  enumerations = 1
stage 2:  enumerations = 27
stage 1:  enumerations = 0

           |        Married
      Race |    Single    Married |     Total
-----------+----------------------+----------
     White |       487      1,150 |     1,637 
     Black |       309        274 |       583 
     Other |         8         18 |        26 
-----------+----------------------+----------
     Total |       804      1,442 |     2,246 

           Fisher's exact =                 0.000


## Notices

在一般的统计学教程或书籍中，有如下的说明：

1. 总例数≥40，所有理论频数≥5，看Pearson Chi-Square结果;
2. 总例数≥40，出现1个理论频数≥1且<5，$\chi^2$检验需进行连续性校正，这时以**Continuity Correction**结果为准
3. 总例数≥40，至少2个理论频数≥1且<5，看**Fisher's Exact Test**结果;
4. 总例数<40或者出现理论频数<1，看**Fisher’s Exact Test**结果

针对上述 第2点 中对$\chi^2$检验需进行连续性校正，在 Stata 中一般不是必须的：

1. Stata不自带卡方检验的连续性矫正
2. Stata有用户自写的package可以实现连续性矫正，但是并不推荐:卡方检验的连续性矫正并不是必须的、也不是最推荐的方法
3. 在样本量足够大的时候，使用卡方检验时，是否使用卡方检验的连续性校正区别很小;使用Fisher精确检验也是没有问题的
4. 在样本量小的时候(通常是某个格子期望频<5)，可直接使用Fisher精确检验，亦不需要使用“卡方检验+连续性矫正”